# R 17 Create Centre of polygons 
- for backend

In [1]:
import geopandas as gpd
import pandas as pd
import json

/Applications/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
Election = pd.read_csv('../data/curated/ElectionData/2022ElectionData.csv')
Election.sort_values(['DivisionName'])

,Unnamed: 0,state,DivisionId,DivisionAb,DivisionName,WinningParty2022,WinningPercent2022,WinningParty2019,WinningPercent2019,ChangedWinningParty,ChangeInWinningPercent,2022WinningPartyOwnChangeInPercent
69,69,SA,179.0,ADEL,Adelaide,ALP,0.581838,ALP,0.583035,False,-0.001197,-0.001197
82,82,VIC,197.0,ASTO,Aston,LP,0.601259,LP,0.574094,False,0.027165,0.027165
83,83,VIC,198.0,BALL,Ballarat,ALP,0.602348,ALP,0.573600,False,0.028748,0.028748
2,2,NSW,103.0,BANK,Banks,LP,0.562619,LP,0.514405,False,0.048214,0.048214
70,70,SA,180.0,BARK,Barker,LP,0.689446,LP,0.534262,False,0.155184,0.155184
...,...,...,...,...,...,...,...,...,...,...,...,...
44,44,NSW,153.0,WERR,Werriwa,ALP,0.554712,ALP,0.581965,False,-0.027253,-0.027253
41,41,NSW,150.0,WHIT,Whitlam,ALP,0.609140,ALP,0.637234,False,-0.028095,-0.028095
68,68,QLD,178.0,WBAY,Wide Bay,LNP,0.631488,LNP,0.581880,False,0.049608,0.049608
112,112,VIC,234.0,WILL,Wills,ALP,0.584651,ALP,0.549277,False,0.035374,0.035374


## Get 2019 and 2022 electorate list

In [3]:
df_2019 = Election[(Election['WinningParty2019'] == 'LP') | (Election['WinningParty2019'] == 'LNP') | (Election['WinningParty2019'] == 'NP')]
df_2022 = Election[Election['WinningParty2022'] == 'ALP']

In [4]:
winning_electorate_2019 = list(df_2019['DivisionName'])
winning_electorate_2022 = list(df_2022['DivisionName'])

## Get centroids from geographic data 

In [5]:
gdf = gpd.read_file('../data/raw/Geography/2021_ELB_region.shp')
gdf = gdf[['Elect_div', 'geometry']]

In [6]:
# deal with centroid of multipolygons: create the right centroid point
centroids = []
for index, row in gdf.iterrows():
    if row['geometry'].geom_type == 'MultiPolygon':
        largest_polygon = max(row['geometry'], key=lambda geom: geom.area)
        centroid = largest_polygon.centroid
    else:
        centroid = row['geometry'].centroid
    
    centroids.append(centroid)


/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_63466/601049228.py:5: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  largest_polygon = max(row['geometry'], key=lambda geom: geom.area)


In [7]:
gdf['centroid'] = centroids

/Applications/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [9]:
gdf = gdf[['Elect_div', 'centroid']]

## Get it into Geolocation form

In [10]:
outlist2022 = []
outlist2019 = []

for id, row in gdf.iterrows():

    if row['Elect_div'] in winning_electorate_2022:
        outlist2022.append({'lat': row['centroid'].coords[0][0], 'lng': row['centroid'].coords[0][1], 'name':row['Elect_div'] })
    
    if row['Elect_div'] in winning_electorate_2019:
        outlist2019.append({'lat': row['centroid'].coords[0][0], 'lng': row['centroid'].coords[0][1], 'name':row['Elect_div']})

In [11]:
with open('../data/curated/GeoJson/2019_winning_party_centroid.json', 'w') as f:
    json.dump(outlist2019, f)

with open('../data/curated/GeoJson/2022_winning_party_centroid.json', 'w') as f:
    json.dump(outlist2022, f)